In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import cv2
import json
from google.colab.patches import cv2_imshow
import pprint
import os
import random
pp = pprint.PrettyPrinter(indent=1)
extra=0

In [ ]:
f_json={'images':[],'annotations':[],'categories': [{'color': '#34d34d',
                 'id': 1,
                 'keypoint_colors': ['#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000'],
                 'keypoints': ['top_front_left',
                               'top_front_right',
                               'top_back_left',
                               'top_back_right',
                               'bot_front_left',
                               'bot_front_right',
                               'bot_back_left',
                               'bot_back_right'],
                 'metadata': {},
                 'name': 'forklift',
                 'skeleton': [[1, 2],
                              [1, 3],
                              [1, 5],
                              [2, 4],
                              [2, 6],
                              [3, 7],
                              [3, 4],
                              [4, 8],
                              [5, 6],
                              [5, 7],
                              [6, 8],
                              [7, 8]],
                 'supercategory': ''},
                {'color': '#c28e00',
                 'id': 3,
                 'keypoint_colors': ['#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000',
                                     '#000'],
                 'keypoints': ['nose',
                               'left_eye',
                               'right_eye',
                               'left_ear',
                               'right_ear',
                               'left_shoulder',
                               'right_shoulder',
                               'left_elbow',
                               'right_elbow',
                               'left_wrist',
                               'right_wrist',
                               'left_hip',
                               'right_hip',
                               'left_knee',
                               'right_knee',
                               'left_ankle',
                               'right_ankle'],
                 'metadata': {},
                 'name': 'person',
                 'skeleton': [[1, 2],
                              [1, 3],
                              [2, 3],
                              [2, 4],
                              [3, 5],
                              [4, 6],
                              [5, 7],
                              [6, 12],
                              [6, 7],
                              [6, 8],
                              [7, 13],
                              [7, 9],
                              [8, 10],
                              [9, 11],
                              [12, 14],
                              [12, 13],
                              [13, 15],
                              [14, 16],
                              [15, 17]],
                 'supercategory': ''}]}

In [ ]:
f = open('drive/MyDrive/TRTPose/data.json')
data = json.load(f)

In [ ]:
pp.pprint(data)

In [ ]:
for img in data['images']:
  im=cv2.imread('drive/MyDrive/TRTPose/data_to_crop/'+img['file_name'])
  extra=50
  if(not(im is None)):
    for ann in data['annotations']:
      if(ann['image_id']==img['id'] and ann['category_id']==1):
        #pp.pprint(ann)
        if((ann['bbox'][1]-extra)<0 or (ann['bbox'][0]-extra)<0):
          extra=0
        crop_im=im[ann['bbox'][1]-extra:ann['bbox'][1]+ann['bbox'][3]+extra,ann['bbox'][0]-extra:ann['bbox'][0]+ann['bbox'][2]+extra]
        cv2.imwrite('drive/MyDrive/TRTPose/Extra/dataset/'+img['file_name'][:-4]+'_Crop.jpg',crop_im)
        #cv2_imshow(crop_im)
        seg=ann['segmentation']
        for i in range(0,len(seg)):
          se=np.array(seg[i],dtype=np.int32)
          se=np.resize(se, (len(se)//2,2))
          se=se-(ann['bbox'][0]-extra,ann['bbox'][1]-extra)
          se=np.resize(se, len(se)*2)
          ann['segmentation'][i]=[float(i) for i in list(se)]
        kp=ann['keypoints']
        kp=np.array(kp,dtype=np.int32)
        kp=np.resize(kp, (len(kp)//3,3))
        kp=kp-(ann['bbox'][0]-extra,ann['bbox'][1]-extra,0)
        kp=np.resize(kp, len(kp)*3)
        ann['keypoints']=[float(i) for i in list(kp)]
        x=ann['bbox'][0]
        y=ann['bbox'][1]
        ann['bbox'][0]=extra
        ann['bbox'][1]=extra
        f_json['annotations'].append(ann)
        for ann_per in data['annotations']:
          if(ann_per['image_id']==img['id'] and ann_per['category_id']==3):
            seg=ann_per['segmentation']
            for i in range(0,len(seg)):
              se=np.array(seg[i],dtype=np.int32)
              se=np.resize(se, (len(se)//2,2))
              se=se-(x-extra,y-extra)
              for j in range(0,len(se)):
                if(se[j][0]<0):
                  se[j][0]=0
                if(se[j][0]>crop_im.shape[1]):
                  se[j][0]=crop_im.shape[1]
                if(se[j][1]<0):
                  se[j][1]=0
                if(se[j][1]>crop_im.shape[0]):
                  se[j][1]=crop_im.shape[0]
              se=np.resize(se, len(se)*2)
              ann_per['segmentation'][i]=[float(i) for i in list(se)]
            kp=ann_per['keypoints']
            kp=np.array(kp,dtype=np.int32)
            kp=np.resize(kp, (len(kp)//3,3))
            kp=kp-(x-extra,y-extra,0)
            for j in range(0,len(kp)):
              if(kp[j][0]<0):
                kp[j][0]=0
              if(kp[j][0]>crop_im.shape[1]):
                kp[j][0]=crop_im.shape[1]
              if(kp[j][1]<0):
                kp[j][1]=0
              if(kp[j][1]>crop_im.shape[0]):
                kp[j][1]=crop_im.shape[0]
            kp=np.resize(kp, len(kp)*3)
            ann_per['keypoints']=[float(i) for i in list(kp)]
            ann_per['bbox'][0]=ann_per['bbox'][0]-(x-extra)
            ann_per['bbox'][1]=ann_per['bbox'][1]-(y-extra)
            f_json['annotations'].append(ann_per)
    img['file_name']=img['file_name'][:-4]+'_Crop.jpg'
    img['width']=crop_im.shape[1]
    img['height']=crop_im.shape[0]
    f_json['images'].append(img)

In [ ]:
with open("drive/MyDrive/TRTPose/Extra/crop_data.json", "w") as outfile:
    json.dump(f_json, outfile)

Negative samples

In [ ]:
path=os.listdir('drive/MyDrive/TRTPose/data_to_crop')
print(len(path))

400


In [ ]:
for i in path:
  im=cv2.imread('drive/MyDrive/TRTPose/data_to_crop/'+i)
  x1=random.randint(0, 1000)
  y1=random.randint(0, 2000)
  h=random.randint(200, 350)
  w=random.randint(200, 350)
  im=im[x1:x1+w,y1:y1+h]
  cv2.imwrite('drive/MyDrive/TRTPose/Extra/neg_dataset/'+i[:-4]+'neg_Crop1.jpg',im)

In [ ]:
path=os.listdir('drive/MyDrive/TRTPose/Extra/neg_dataset')
print(len(path))

400
